# Criando usuários na API

In [49]:
import json
import random
import requests
import pandas as pd

In [39]:

url = 'https://sdw-2023-prd.up.railway.app/users'
nomes = ['Pedro', 'Ana', 'João', 'Victor', 'Renato', 'Julia', 'Luana', 'Luis']
sobrenomes = ['Cardoso', 'Coelho', 'Moura', 'Silva', 'Peixoto']
ids = []

for _ in range(10):
    full_name = nomes[random.randint(0,len(nomes)-1)] + ' ' + sobrenomes[random.randint(0,len(sobrenomes)-1)]
    account_number = f'{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}-{random.randint(0,9)}'
    agency = f'{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}'
    card_number = f'{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)} '
    card_number += f'{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)} '
    card_number += f'{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)} '
    card_number += f'{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}'
                
    obj = {
    "name": full_name,
    "account": {
        "number": account_number,
        "agency": agency,
        "balance": random.randint(0,100000),
        "limit": random.randint(0,100000)
    },
    "card": {
        "number": card_number,
        "limit": random.randint(0,10000)
    }
    }
    response = requests.post(url, json = obj)
    response = json.loads(response.text)
    ids.append(response.get('id'))
    

In [47]:
df = pd.DataFrame({'UserID':ids})
df.to_csv('UserID.csv')


# Extract

In [1]:
import json
import requests
import pandas as pd
from typing import List
from typing import Any
from dataclasses import dataclass

In [2]:
df = pd.read_csv('UserID.csv')
ids = df['UserID'].to_list()

In [3]:
@dataclass
class Account:
    id: int
    number: str
    agency: str
    balance: float
    limit: float

    @staticmethod
    def from_dict(json):
        id = int(json.get('id'))
        number = str(json.get('number'))
        agency = str(json.get('agency'))
        balance = float(json.get('balance'))
        limit = float(json.get('limit'))
        return Account(id, number, agency, balance, limit)

@dataclass
class Card:
    id: int
    number: str
    limit: float

    @staticmethod
    def from_dict(json):
        id = int(json.get('id'))
        number = str(json.get('number'))
        limit = float(json.get('limit'))
        return Card(id, number, limit)

@dataclass
class Features:
    id: int
    icon: str
    description: str

    @staticmethod
    def from_dict(json):
        id = int(json.get('id'))
        icon = str(json.get('icon'))
        description = str(json.get('description'))
        return Features(id, icon, description)

@dataclass
class News:
    id: int
    icon: str
    description: str
   
    @staticmethod
    def from_dict(json):
        id = int(json.get('id'))
        icon = str(json.get('icon'))
        description = str(json.get('description'))
        return News(id, icon, description)

@dataclass
class User:
    id: int
    name: str
    account: Account
    card: Card
    features = List[Features]
    news = List[News]

    def __init__(self, _id, _name, _account, _card, _features, _news):
        self.id = _id
        self.name = _name
        self.account = _account
        self.card = _card
        self.features = _features
        self.news = _news

    @staticmethod
    def from_dict(json):
        id = int(json.get('id'))
        name = str(json.get('name'))
        account = Account.from_dict(json.get('account'))
        card = Card.from_dict(json.get('card'))
        features = [Features.from_dict(feature) for feature in json.get('features')]
        news = [News.from_dict(new) for new in json.get('news')]
        return User(id, name, account, card, features, news)
    
    @staticmethod
    def get_users(ids):
        endpoint = 'https://sdw-2023-prd.up.railway.app/users'
        users = []
        for id in ids:
            if id is not None:
                response = requests.get(f'{endpoint}/{id}')
                if response.status_code == 200:
                    print(id)
                    user = json.loads(json.dumps(response.json()))
                    users.append(User.from_dict(user))
        
        return users
    

In [4]:
users = User.get_users(ids)

3293
3294
3295
3296
3297
3298
3299
3300
3301
3302


# Transform

In [ ]:

!pip install openai

In [5]:
import openai
import time

openai.api_key = ''

In [6]:
def get_news(users):
    for user in users:
        completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Você é um especialista em marketing bancário."},
            {"role": "user", "content": f"Criei uma mensagem para {user.name} sobre a importância da educação financeira e de investir (máximo de 200 caracteres)."}
        ]
        )
        user.news.append(News(None,'https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg',completion.choices[0].message.content.strip('\"')))
        time.sleep(25)
    return users

In [7]:
def get_features(users):
    for user in users:
        completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Você é um especialista em marketing bancário."},
            {"role": "user", "content": f"Criei uma sugestão de investimento para o cliente {user.name}. Saldo de {user.account.balance} e um limite de credito de {user.account.limit} (máximo de 200 caracteres)."}
        ]
        )
        user.features.append(Features(None,'https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg',completion.choices[0].message.content.strip('\"')))
        time.sleep(25)
    return users

In [8]:
users = get_news(users)



In [9]:
users = get_features(users)

In [10]:
users[0].news

[News(id=6836, icon='https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg', description='Julia, invista em você mesma! Educação financeira é o caminho para uma vida financeira saudável e próspera. Não perca essa oportunidade!'),
 News(id=None, icon='https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg', description='Julia, invista em você mesma e em seu futuro financeiro. Aprenda a ter controle sobre seu dinheiro e faça ele trabalhar para você. Educação financeira é o caminho para o sucesso!')]

In [11]:
users[3].features

[Features(id=None, icon='https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg', description='Sugestão de investimento para o cliente Luis Moura: Diversifique seu portfólio de investimentos com uma combinação de renda fixa e variável para maximizar seus retornos. Considere investir uma parte em títulos de renda fixa de baixo risco e o restante em ações de empresas sólidas e com bom potencial de crescimento. Aproveite seu saldo de 82857.0 e limite de crédito de 76452.0 para aumentar suas oportunidades de investimento.')]

# Load

In [12]:
#for user in users:
jsonStr = json.dumps(users[0].__dict__, default=lambda o: o.__dict__, indent=4)

In [13]:
json.loads(jsonStr)

{'id': 3293,
 'name': 'Julia Peixoto',
 'account': {'id': 3491,
  'number': '8999-1',
  'agency': '5605',
  'balance': 78326.0,
  'limit': 43503.0},
 'card': {'id': 3187, 'number': '3451 5609 6809 5131', 'limit': 1236.0},
 'features': [{'id': 1050,
   'icon': 'https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg',
   'description': 'Sugestão de investimento para Julia Peixoto: Considerando seu saldo de R$78.326,00 e limite de crédito de R$43.503,00, recomendo diversificar seus investimentos. Aproveite os benefícios da renda fixa,'},
  {'id': None,
   'icon': 'https://github.com/digitalinnovationone/santander-dev-week-2023-api/blob/main/docs/icons/credit.svg',
   'description': 'Sugestão de Investimento para Julia Peixoto: Diversifique seu portfólio com uma combinação de fundos de renda fixa e ações, equilibrando segurança e potencial de retorno. Agende uma consulta para discutirmos estratégias sob medida para seu perfil. #InvestimentoPerso

In [ ]:
endpoint = 'https://sdw-2023-prd.up.railway.app'

for user in users:
    obj = {
    "id": user.id,
    "name": user.name,
    "account": {
        "id": user.account.id,
        "number": user.account.number,
        "agency": user.account.agency,
        "balance": user.account.balance,
        "limit": user.account.limit
    },
    "card": {
        "id": user.card.id,
        "number": user.card.number,
        "limit": user.card.limit
    },
    "features": 
        [{"icon": feature.icon,"description": feature.description[:200]} for feature in user.features]
    ,
    "news": [{"icon": new.icon,"description": new.description[:200]} for new in user.news]
    }
    response = requests.put(f'{endpoint}/users/{user.id}', data = (json.dumps(obj)))
    
    if response.status_code == 200:
        print(f'Usuario: {user.id} atualizado com sucesso!')
        print(response)
    else:
        print(f'Usuario: {user.id} não foi atualizado com sucesso!')
        print(json.dumps(obj))
        print(response.content)